## **Очистка данных**

In [ ]:
import numpy as np
import pandas as pd

Будем использовать немного видоизменённый для наших целей датасет https://archive.ics.uci.edu/ml/datasets/AutoUniv.

В нём присутствуют целочисленные, вещественнозначные и категориальные признаки.

In [ ]:
df = pd.read_csv('table.csv')
df.head()

Все нечисловые признаки имеют тип object. Найдем их в нашей таблице.

In [ ]:
#Your code is here
cat_features_mask = ...

In [ ]:
len(cat_features_mask[cat_features_mask==True])

**Заполнение пропусков**

В исходных данных могут быть пропущенные значения. Большинство методов машинного обучения не умеют с ними работать. Для этого мы должны каким-нибудь образом заполнить эти пропуски.

Способы заполнения пропусков:

* средним значением 
* медианой
* самым часто встречающимся значением
* каким-то одним новым значением (иногда пропуск в данных можно воспринимать как еще одно категориальное значение).
* можно взять часть данных для обучения, а на другой части данных предсказать пропущенные значения (тогда для решения основной задачи нельзя обучаться на первой части данных)

Посмотрим, есть ли пропуски в наших данных и много ли их.

In [ ]:
#индексы строк с NAN

df_real = df[df.columns[~cat_features_mask]]

print np.any(np.isnan(df_real))

gaps = pd.isnull(df_real).any(1).nonzero()[0]
print gaps

**Удалим все строки, содержащие пропущенные значения.**

In [ ]:
#Your code is here

df_drop_na = ...

**Заполним пропуски в данных с помощью встроенного метода Imputer, используя среднее значение признака.**

In [ ]:
from sklearn import preprocessing

mis_replacer = preprocessing.Imputer(strategy="mean")
df_no_mis = pd.DataFrame(data=mis_replacer.fit_transform(df_real))

In [ ]:
np.any(np.isnan(df_no_mis))

Посмотрим на столбец 'att2'.

In [ ]:
df = pd.read_csv('table.csv')

df['att2'][:10]

**'att2' - время в минутах, в течение которого в машинах работал специально установленный датчик особого типа. Как заполнить пропуски в 'att2'? (этого датчика может в машине не быть)**

In [ ]:
#Your code is here

**Работа с выбросами**

In [ ]:
%pylab inline

data = df['att3'].fillna(df['att3'].mean()).values
plt.hist(data)

In [ ]:
np.median(data), np.percentile(data,5), np.percentile(data,95)

In [ ]:
print sort(data)

In [ ]:
data_no_outliers = sort(data)[1:-1]
hist(data_no_outliers)

Попробуем найти аутлаеры с помощью sklearn.

In [ ]:
from sklearn.ensemble import IsolationForest

Xdata = [[elem] for elem in data]

#max_samples = length of data and contamination is a fraction of outliers in (0,1) 
clf = IsolationForest(max_samples=..., contamination=...)

#fit and predict Xdata
#Your code is here

#print outliers
#Your code is here

**Пример из sklearn по удалению аутлаеров**

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager

from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

print(__doc__)

rng = np.random.RandomState(42)

# Example settings
n_samples = 200
outliers_fraction = 0.25
clusters_separation = [0, 1, 2]

# define two outlier detection tools to be compared
classifiers = {
    "One-Class SVM": svm.OneClassSVM(nu=0.95 * outliers_fraction + 0.05,
                                     kernel="rbf", gamma=0.1),
    "Robust covariance": EllipticEnvelope(contamination=outliers_fraction),
    "Isolation Forest": IsolationForest(max_samples=n_samples,
                                        contamination=outliers_fraction,
                                        random_state=rng)}

# Compare given classifiers under given settings
xx, yy = np.meshgrid(np.linspace(-7, 7, 100), np.linspace(-7, 7, 100))
n_inliers = int((1. - outliers_fraction) * n_samples)
n_outliers = int(outliers_fraction * n_samples)
ground_truth = np.ones(n_samples, dtype=int)
ground_truth[-n_outliers:] = -1

# Fit the problem with varying cluster separation
for i, offset in enumerate(clusters_separation):
    np.random.seed(42)
    # Data generation
    X1 = 0.3 * np.random.randn(n_inliers // 2, 2) - offset
    X2 = 0.3 * np.random.randn(n_inliers // 2, 2) + offset
    X = np.r_[X1, X2]
    # Add outliers
    X = np.r_[X, np.random.uniform(low=-6, high=6, size=(n_outliers, 2))]

    # Fit the model
    plt.figure(figsize=(9, 7))
    for i, (clf_name, clf) in enumerate(classifiers.items()):
        # fit the data and tag outliers
        if clf_name == "Local Outlier Factor":
            y_pred = clf.fit_predict(X)
            scores_pred = clf.negative_outlier_factor_
        else:
            clf.fit(X)
            scores_pred = clf.decision_function(X)
            y_pred = clf.predict(X)
        threshold = stats.scoreatpercentile(scores_pred,
                                            100 * outliers_fraction)
        n_errors = (y_pred != ground_truth).sum()
        # plot the levels lines and the points
        if clf_name == "Local Outlier Factor":
            # decision_function is private for LOF
            Z = clf._decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        subplot = plt.subplot(2, 2, i + 1)
        subplot.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7),
                         cmap=plt.cm.Blues_r)
        a = subplot.contour(xx, yy, Z, levels=[threshold],
                            linewidths=2, colors='red')
        subplot.contourf(xx, yy, Z, levels=[threshold, Z.max()],
                         colors='orange')
        b = subplot.scatter(X[:-n_outliers, 0], X[:-n_outliers, 1], c='white',
                            s=20, edgecolor='k')
        c = subplot.scatter(X[-n_outliers:, 0], X[-n_outliers:, 1], c='black',
                            s=20, edgecolor='k')
        subplot.axis('tight')
        subplot.legend(
            [a.collections[0], b, c],
            ['learned decision function', 'true inliers', 'true outliers'],
            prop=matplotlib.font_manager.FontProperties(size=10),
            loc='lower right')
        subplot.set_xlabel("%d. %s (errors: %d)" % (i + 1, clf_name, n_errors))
        subplot.set_xlim((-7, 7))
        subplot.set_ylim((-7, 7))
    plt.subplots_adjust(0.04, 0.1, 0.96, 0.94, 0.1, 0.26)
    plt.suptitle("Outlier detection")

plt.show()

**Масштабирование признаков**

Для применения в особенности линейных методов машинного обучения масштабирование признаков очень важно. Может быть так, что метод даст совершенно неправильный результат без масштабирования.

Масштабирование признаков можно выполнить, например, одним из следующих способов способами:
 - $x_{new} = \dfrac{x - \mu}{\sigma}$, где $\mu, \sigma$ — среднее и стандартное отклонение значения признака по всей выборке (StandardScaler в sklearn)
 - $x_{new} = \dfrac{x - x_{min}}{x_{max} - x_{min}}$, где $[x_{min}, x_{max}]$ — минимальный интервал значений признака (MinMaxScaler в sklearn)

In [ ]:
normalizer = preprocessing.StandardScaler()
df_real_norm = normalizer.fit_transform(df_no_mis)
df_real_norm_pd = pd.DataFrame(data=df_real_norm)

In [ ]:
mm_scaler = preprocessing.MinMaxScaler()
df_mm_scaled = mm_scaler.fit_transform(df_no_mis)
df_mm_scaled_pd = pd.DataFrame(data=df_mm_scaled)

Разберем пример данных, когда масштабирование признаков сильно влияет на результат работы алгоритмов машинного обучения. Рассмотрим точки, равномерно нанесенные на плоскость (в данном датасете в качестве признаков x и y выступают координаты - долгота и широта точек, все точки территориально находятся в Москве).

In [6]:
X = pd.read_csv('scaler_example.csv')
X.head()

Посмотрим, как выглядят точки из датасета. Точки относятся к двум классам.

In [4]:
%pylab inline

Xtrain = X[['lat','lon']].values
Ytrain = X['class'].values

#Plot points of different classes with different colours on one image

figure(figsize=(10,10))
#Your code is here

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(Xtrain,Ytrain)

def plot_decision_line(Xtrain, Ytrain, clf, h):

    x_min, x_max = Xtrain[:,0].min()-0.01, Xtrain[:,0].max()+0.01
    y_min, y_max = Xtrain[:,1].min()-0.01, Xtrain[:,1].max()+0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(10,10))
    plt.contourf(xx, yy, Z)

    Ytrain = np.array(Ytrain)
    plt.scatter(Xtrain[Ytrain==1][:,0],Xtrain[Ytrain==1][:,1],color='green')
    plt.scatter(Xtrain[Ytrain==0][:,0],Xtrain[Ytrain==0][:,1],color='red')

    a = clf.coef_[0][0]
    b = clf.coef_[0][1]
    c = clf.intercept_

    K = -a*1./b
    B = -c*1./b

    xx0 = np.linspace(x_min, x_max)
    yy0 = K * xx0 + B

    plt.scatter(xx0,yy0)
    plt.show()
    
plot_decision_line(Xtrain, Ytrain, clf, 0.001)

   Мы видим, что SVM абсолютно не справился с задачей. Попробуем теперь масштабировать данные.

In [ ]:
from sklearn.preprocessing import StandardScaler

#apply scaler to data and fit classifier
#Your code is here

In [ ]:
#plot new decision line
#Your code is here